In [212]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from transformers import 

import tensorflow as tf
import pprint

In [213]:
data_path = "../datasets/clean/final.npy"
data = np.load(data_path)

- Import best model (trained on kaggle GPUs) to make predictions

In [217]:
import re

def pre_process(text):
    
    text = text.replace("\n", "")
    text = text.lower()
    text = text.strip()
    # text = text.replace(" ", "")
    
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '<EMAIL>', text)
    text = re.sub(r'\b\w+(?:\.\w+)+\b', '<SUB>', text)
    text = re.sub(r'(\d+:)+\d+', '<TIME>', text)
    text = re.sub(r'\d{4}-\d{2}-\d{2}', '<DATE>', text)
    text = re.sub(r'\b(\w+)(?:,\1)+\b', '<SERIES>', text)
    text = re.sub(r'\b\d+(?:,\d+)+\b', '<SERIES>', text)
    text = re.sub(r'\bchar\(\d+(?:\+\d+)*\)', '<SERIES>', text)
    text = re.sub(r'<SERIES>(?:\+<SERIES>)+', '<SERIES>', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\b', '<NUMBER>', text)
    text = re.sub(r'(.)\1{2,}', '<REPETITIVE>', text)
    text = re.sub(r'(?<=[@$%^!~/[\]\\` ])(?!a)\w(?=[@$%^!~/[\]\\` ])', '<SINGLE>', text)
    
    special_characters = r'[@$%^!~/[\]\-\`]'
    text = re.sub(rf'{special_characters}{{2,}}', '<REGEX>', text)
    text = re.sub(special_characters, '<SPECIAL>', text)

    text = text.replace(",", "")

    return text

In [218]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

def encode(query: str, tokenizer) -> list[int]:
    query = pre_process(query)
    tok = tokenizer.tokenize(query)
    tok = list(map(lambda x: x.replace("Ġ", ""), tok))
    
    for i, t in enumerate(tok):
        if bool(re.fullmatch(r'[a-zA-Z]', t)) or bool(re.fullmatch(r'-?\d+', t)):
            tok[i] = "<oov>"
            
    seq = tokenizer.convert_tokens_to_ids(tok)
    seq = [item for item in seq if item != None]
    act_len = len(seq)
    seq = pad_sequences(sequences=[seq], 
                              maxlen=15, 
                              padding="post", 
                              truncating="post")
    return (seq, act_len)

In [226]:
def predict_proba(sentence:str, tokenizer, model) -> float:
    (sequence, act_len) = encode(sentence, tokenizer)
    sentence = np.expand_dims(sentence, axis=0)
    pred = model.predict(sequence)
    seq_pred = np.argmax(pred, axis=2)
    proba = np.sum(seq_pred[0, :act_len] == sequence[0, :act_len]) / act_len
    
    return proba

In [228]:
from transformers import GPT2Tokenizer
vocab = "../utils/tokenier/vocab/vocab.json"
merges = "../utils/tokenier/vocab/merges.txt"
tokenizer = GPT2Tokenizer(vocab_file=vocab, merges_file=merges)

TypeError: GPT2Tokenizer.__init__() missing 1 required positional argument: 'merges_file'

In [221]:
model_path = "../utils/best_models/tr_acc_8974_val_acc_8549.keras"
model = tf.keras.models.load_model(model_path)

In [227]:
inp = [
    "' AND (SELECT COUNT(*) FROM information_schema.columns WHERE table_name='users') > 0 --",
    "' AND (SELECT COUNT(*) FROM users) > 0 --",
    "' AND 1=1 ORDER BY 2 --",
    "' AND (SELECT SUBSTRING(password,1,1) FROM users WHERE username='admin')='h' --",
    "SELECT DISTINCT country FROM customers ORDER BY country;"
]

for q in inp:
    print(predict_proba(q, tokenizer=tokenizer, model=model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
0.17857142857142858
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
0.21052631578947367
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
0.14285714285714285
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
0.34615384615384615
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
0.6666666666666666
